In [1]:
import keras
import pandas as pd
import numpy as np
from joblib import load

In [2]:
model_single = keras.models.load_model("temp/first_model_checkpoint.h5")
model_pitch = keras.models.load_model("temp/pitch_model_checkpoint.h5")
model_duration = keras.models.load_model("temp/duration_model_checkpoint.h5")
model_time2next = keras.models.load_model("temp/time2next_model_checkpoint.h5")

duration_scaler = load("data/duration_scaler.save")
pitch_scaler = load("data/pitch_scaler.save")
time_to_next_scaler = load("data/time_to_next_scaler.save")

In [3]:
import pretty_midi
import glob

In [4]:
initial_midi = pretty_midi.PrettyMIDI("../midi/for_creation_midi/schubert_sonata_157_2_(c)yogore.mid")
print([ (instrument, len(instrument.notes)) for instrument in initial_midi.instruments ])

[(Instrument(program=0, is_drum=False, name="Grand Piano"), 1309), (Instrument(program=0, is_drum=False, name="Instrument17"), 301), (Instrument(program=0, is_drum=False, name="Instrument18"), 1066), (Instrument(program=0, is_drum=False, name="Instrument19"), 225)]


In [5]:
midi_files = glob.glob(r"..\midi\for_creation_midi\*")
initial_dict = {}

for midi_file in midi_files:
    initial_midi = pretty_midi.PrettyMIDI(midi_file)

    treated_midi = pretty_midi.PrettyMIDI()
    treated_piano = pretty_midi.Instrument(
        pretty_midi.instrument_name_to_program('Acoustic Grand Piano'),
        name = 'Piano'
    )
    treated_piano.notes = []
    treated_midi.instruments = []
    valid_instruments = [instrument for instrument in initial_midi.instruments \
        if pretty_midi.program_to_instrument_class(instrument.program) in ["Piano", "Organ"]]
    for instrument in valid_instruments:
        for note in instrument.notes:
            note.velocity = 60 # all notes will have the same velocity, as it is not really relevant for structure
            treated_piano.notes.append(note)
    treated_midi.instruments.append(treated_piano)

    song_dict = {
        "start": [],
        "end": [],
        "pitch": [],
    }
    for note in initial_midi.instruments[0].notes:
        song_dict["start"].append(note.start)
        song_dict["end"].append(note.end)
        song_dict["pitch"].append(note.pitch)
    song_dict = pd.DataFrame(song_dict).sort_values(by=['start', 'end'], ignore_index=True)

    song_dict['duration'] = song_dict['end'] - song_dict['start']

    time_to_next = []
    for note_idx in range(song_dict.shape[0]):
        if note_idx == (song_dict.shape[0]-1):
            value = 0
        else:
            value = song_dict.at[note_idx+1, 'start'] - song_dict.at[note_idx, 'start']
        time_to_next.append(value)
    song_dict['time_to_next'] = time_to_next

    note_duration = pd.Series(dtype="float64")
    time_to_next = pd.Series(dtype="float64")

    song_dict.loc[:, 'duration'] = np.log2(song_dict.duration)
    song_dict.loc[:, 'time_to_next'] = np.log(song_dict.time_to_next + 1)/np.log(100) ## needed to add 1 to avoid 0 value for the log
    time_to_next = pd.concat([time_to_next, song_dict['time_to_next']])
    note_duration = pd.concat([note_duration, song_dict['duration']])

    pitch_sc = pitch_scaler.transform(np.array(song_dict['pitch']).reshape(-1,1)).flatten()
    time_to_next_sc = time_to_next_scaler.transform(np.array(time_to_next[:]).reshape(-1,1)).flatten()
    duration_sc = duration_scaler.transform(np.array(note_duration[:]).reshape(-1,1)).flatten()

    initial_dict[midi_file] = {
        "pitch": pitch_sc[:10],
        "duration": duration_sc[:10],
        "time2next": time_to_next_sc[:10]
    }

In [6]:
print(list(initial_dict.items())[0])

('..\\midi\\for_creation_midi\\schubert_sonata_157_2_(c)yogore.mid', {'pitch': array([0.54761905, 0.57142857, 0.54761905, 0.53571429, 0.57142857,
       0.5952381 , 0.57142857, 0.54761905, 0.53571429, 0.54761905]), 'duration': array([0.52124202, 0.28576063, 0.43433285, 0.66981424, 0.52124202,
       0.28576063, 0.43433285, 0.66981424, 0.52124202, 0.28576063]), 'time2next': array([0.25287558, 0.10083267, 0.18321896, 0.41404755, 0.25287558,
       0.10083267, 0.18321896, 0.41404755, 0.25287558, 0.10083267])})


In [7]:
created_midi_data_single_model = {}
created_midi_data_mult_model = {}

for name, data in initial_dict.items():
    pitch, duration, time_to_next = data.values()
    for i in range(250):
        input = [list(zip(pitch[-10:], duration[-10:], time_to_next[-10:]))]
        pred_note = model_single.predict(input)
        pitch = np.append(pitch, pred_note[0][0])
        duration = np.append(duration, pred_note[0][1])
        time_to_next = np.append(time_to_next, pred_note[0][2])

    created_midi_data_single_model[name] = {
        "pitch": pitch_scaler.inverse_transform(pitch.reshape(-1, 1)),
        "duration": np.power(2, duration_scaler.inverse_transform(duration.reshape(-1, 1))),
        "time2next": np.power(100, time_to_next_scaler.inverse_transform(time_to_next.reshape(-1, 1))) - 1
    }

    pitch, duration, time_to_next = data.values()
    for i in range(250):
        input = [list(zip(pitch[-10:], duration[-10:], time_to_next[-10:]))]
        pred_pitch = model_pitch.predict(input)[0][0]
        pred_duration = model_duration.predict(input)[0][0]
        pred_time2next = model_time2next.predict(input)[0][0]
        pitch = np.append(pitch, pred_pitch)
        duration = np.append(duration, pred_duration)
        time_to_next = np.append(time_to_next, pred_time2next)

    created_midi_data_mult_model[name] = {
        "pitch": pitch_scaler.inverse_transform(pitch.reshape(-1, 1)),
        "duration": np.power(2, duration_scaler.inverse_transform(duration.reshape(-1, 1))),
        "time2next": np.power(100, time_to_next_scaler.inverse_transform(time_to_next.reshape(-1, 1))) - 1
    }


1/1 [==============================] - 0s 20ms/step


In [8]:
print(list(created_midi_data_single_model.values())[0])

{'pitch': array([[67.        ],
       [69.        ],
       [67.        ],
       [66.        ],
       [69.        ],
       [71.        ],
       [69.        ],
       [67.        ],
       [66.        ],
       [67.        ],
       [64.86204529],
       [65.91981483],
       [67.06209898],
       [66.45613718],
       [66.29019713],
       [65.92396545],
       [66.54892802],
       [65.46849275],
       [64.4229548 ],
       [64.52859306],
       [65.79824996],
       [65.47064066],
       [65.4943378 ],
       [65.01068187],
       [64.89901042],
       [64.86483908],
       [64.77628398],
       [64.67446089],
       [64.53195262],
       [64.43797517],
       [64.37058377],
       [64.22735453],
       [64.11373544],
       [63.99805355],
       [63.87862659],
       [63.7661891 ],
       [63.64270663],
       [63.5428462 ],
       [63.43003821],
       [63.32594204],
       [63.22348809],
       [63.11946201],
       [63.01849508],
       [62.91779602],
       [62.81866908],


In [9]:
for name, data in initial_dict.items():
    pitch, duration, time_to_next = created_midi_data_single_model[name].values()
    created_midi = pretty_midi.PrettyMIDI()
    created_piano = pretty_midi.Instrument(
        pretty_midi.instrument_name_to_program('Acoustic Grand Piano'),
        name = 'Piano'
    )
    created_piano.notes = []
    created_midi.instruments = []
    start_time = 0
    print(len(pitch))
    for idx in range(len(pitch)):
        note = pretty_midi.Note(
            velocity=60,
            pitch=round(pitch[idx][0]),
            start=round(start_time),
            end=round(start_time + duration[idx][0]),
        )
        created_piano.notes.append(note)
        start_time += time_to_next[idx][0]
    created_midi.instruments.append(created_piano)
    created_midi.write(name.replace("for_creation_midi", "created_midi/single_model"))

    pitch, duration, time_to_next = created_midi_data_mult_model[name].values()
    created_midi = pretty_midi.PrettyMIDI()
    created_piano = pretty_midi.Instrument(
        pretty_midi.instrument_name_to_program('Acoustic Grand Piano'),
        name = 'Piano'
    )
    created_piano.notes = []
    created_midi.instruments = []
    start_time = 0
    for idx in range(len(pitch)):
        note = pretty_midi.Note(
            velocity=60,
            pitch=round(pitch[idx][0]),
            start=round(start_time),
            end=round(start_time + duration[idx][0]),
        )
        created_piano.notes.append(note)
        start_time += time_to_next[idx][0]
    created_midi.instruments.append(created_piano)
    created_midi.write(name.replace("for_creation_midi", "created_midi/mult_model"))
    

260
60 67 0 1
60 69 1 1
60 67 1 2
60 66 2 3
60 69 3 4
60 71 4 4
60 69 4 4
60 67 4 6
60 66 6 7
60 67 7 7
60 65 7 7
60 66 7 8
60 67 8 8
60 66 8 8
60 66 8 8
60 66 8 9
60 67 8 9
60 65 9 9
60 64 9 9
60 65 9 10
60 66 9 10
60 65 9 10
60 65 10 10
60 65 10 10
60 65 10 10
60 65 10 10
60 65 10 11
60 65 10 11
60 65 11 11
60 64 11 11
60 64 11 11
60 64 11 11
60 64 11 11
60 64 11 12
60 64 11 12
60 64 12 12
60 64 12 12
60 64 12 12
60 63 12 12
60 63 12 12
60 63 12 12
60 63 12 13
60 63 12 13
60 63 13 13
60 63 13 13
60 63 13 13
60 63 13 13
60 63 13 13
60 62 13 13
60 62 13 13
60 62 13 14
60 62 13 14
60 62 14 14
60 62 14 14
60 62 14 14
60 62 14 14
60 61 14 14
60 61 14 14
60 61 14 14
60 61 14 14
60 61 14 14
60 60 14 14
60 60 14 15
60 60 14 15
60 60 15 15
60 59 15 15
60 59 15 15
60 59 15 15
60 58 15 15
60 58 15 15
60 58 15 15
60 57 15 15
60 57 15 15
60 56 15 15
60 56 15 15
60 55 15 15
60 55 15 15
60 54 15 15
60 54 15 15
60 53 15 15
60 53 15 15
60 52 15 15
60 52 15 15
60 52 15 15
60 52 15 15
60 51 15 15
60 51